In [ ]:
import pandas as pd
import seaborn as sns
import sklearn
from tensorflow import keras

# Convolutional  Neural Networks and Transfer Learning

Today we are going to explore convolutional neural nets (CNNs or ConvNets) for image classification. We'll describe their architecture and components in depth prior to building a simple CNN from scratch in Keras to solve the MNIST digit classification problem. Then we'll talk about how CNNs are most commonly used in practice through transfer learning, where pretrained generalist models are leveraged to efficiently solve more specialized tasks. 

There's a ton of content in this notebook, but a good chunk of it falls under additional considerations / appendix! --

---

1. Thinking About Image Properties
2. Introduction to CNNs
3. Simple CNN: Digit Recognition
4. Transfer Learning with a Convolutional Base
5. Appendix 1: Additional Considerations for Images
6. Appendix 2: Other Deep Learning Architectures for Images

---

### 1. Thinking About Image Properties

An image is made up of pixels, and each pixel has a possible color. There are an insane number of possible arrangements of pixels. for a 512x512 image there are $256^{786432}$ possible arrangments of RGB colors. 

Luckily for us, actual images in the real world aren't equally distributed across all possible images and instead share certain properties that make the problem much more tractable:
* Nearby pixels tend to be similar to each other
* Higher-level patterns build on top of each other
* Pixels form lines and lines form shapes
* It is rare for an image to contain large amounts of static.

**This tendency for nearby pixels to be related means we can exploit this with something called convolutional filters**.

### 2. Introduction to the Structure and Components of CNNs

#### Visual overview of CNNs: 
![](img/conv-net2.png)

#### Convolutions

A **convolution layer** is a specific kind of layer architecture. Whereas fully-connected or dense layers (the kind we saw in our first NN lecture) learn global patterns in the feature space, convolution layers will learn local patterns. This is helpful given the nature of images we cited above. There are two ways in particular that convolutions are so good: 

1. Learned patterns are **translation invariant**: you can flip a chair or move it from the top right to bottom left of an image and it's still a chair. If we recognize the pattern of a chair, we can find that pattern anywhere in an image. This is different from a densely connected layer, which would have to learn a chair in the top right as distinct from a chair in the bottom left. 

2. There's a **hierarchy of features** that a convolution layer will learn: early layers will learn smaller, more local patterns (e.g. edges) and later convolutions will learn patterns that are made up of the earlier patterns! E.g. we can learn that a cat's ears have defined edges, then learn a cat's face as edges of ears plus eyes plus mouth, then learn a full cat as face and body. 

**A convolution works by running a kernel across a layer.** This allows us to learn local patterns. A **stride** is how many pixels you move in either direction (tends to be the same number in both directions). Kernels do not have to be square. 
![](img/3D_Convolution_Animation.gif)

#### Max pooling

We'd have a lot of parameters if we just ran an image through a neural net. The intention of a **max pooling operation is to reduce the feature space**. Max pooling, as its name suggests, takes in a given area of a layer and outputs the maximum. It's similar to running a kernel across a layer as in a convolution above, but in max pooling we tend to only use a 2x2 window and a stride of 2; instead of using one of several types of linear transformations, we use only a max operation. 

![](img/Max_pooling.png)

#### Visualizing What Layers Learn

How to generate these visuals is beyond the scope of this lecture, but they're a nice way to get an intuitive feeling for the heirarchical image pattern extraction process that happens under the hood in a CNN. 

First layer example: 
![](img/filt1.jpeg)

Subsequent layers get harder to visualize ![But Inception networks can give it a go](http://yosinski.com/static/proj/deepvis_all_layers.jpg)

With the core CNN components in mind and a feel for the way that convolutional filters learn patterns, we can **return to the overview image** above to start to piece together the bigger picture. Perhaps even more concretely, we can define a relatively simple keras model that explicitly uses these core components in a very conventional way - let's do that now!  

### 3. Our First Simple CNN: Handwritten Digit Recognition

As usual, we'll need to begin our workflow with some data setup, creating the proper numpy arrays to feed to keras as input and output. Note that when working with keras we'll always treat images geometrically as **3 dimensional tensors** (arrays): **width *x* height *x* color channels**. We'll often have 3 color channels (RGB), but even if we only have 1 (greyscale) channel we'll follow this convention. Our input tensors **X** then must be 4D tensors, since we'll always add a first dimension as the sample dimension (roughly you can think of this as "rows" of images).

In [ ]:
from sklearn.datasets import fetch_openml
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

'''
data setup: load data, 4D tensor formatting, train-test split,
and 2D (one-hot encoded) representation of the training target
'''

mnist = fetch_openml('mnist_784')

X_digits, Y_digits = mnist.data, mnist.target.astype(np.int64)
X_digits = X_digits.reshape((-1,28,28,1)) #28x28 images with only 1 color channel

X_train, X_test, y_train, y_test = (train_test_split(X_digits, Y_digits, 
                                                     test_size = .2, random_state = 42))

y_train_cat = to_categorical(y_train)

With that simple preprocessing taken care of, we have everything we need to construct a CNN model that maps a 4 dimensional input tensor (a collection of 3D images with a color channel dimension) to a 2 dimensional output tensor (collection of one-hot encodings of 10 multiclass targets).

We're introducing several new layer types with adjustable hyperparameters, corresponding to the CNN components described above -- 
* **Conv2D** (convolutions that slide over width and height directions)
     * Confusingly, 2D convolutions aren't 2D because they operate on 2D images, but because they move in 2 directions. In the models in this notebook, they actually operate on 3D tensors with color/filter depth channels, and each filter has one matrix per depth channel that's applied to just that channel. Values of the filter application are then summed across channels to generate 1 channel of output.   
* **MaxPooling2D** (pooling over width and height)
* **Flatten** (simply unravel a 3D tensor, converting it to a long vector of numbers)

With that in mind, let's train a simple model and score it on test data!

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D, InputLayer
'''
 In this network structure, note that we follow the typical CNN heuristic of 
 gradually reducing width and height dimenions over time with max pooling
 (typically by a factor of 2), but increasing the filter depth dimension 
 to find increasingly specific patterns. These models are typically compromised 
 of a series of convolutional blocks followed by a flattening operation and 
 a series of fully connected layers at the terminus.
'''

NN = Sequential()

NN.add(InputLayer(input_shape=X_train.shape[1:]))

# Conv block 1.  You can add more conv steps to
# each block to increase model capacity.
NN.add(Conv2D(filters=10, kernel_size=3, activation='relu', padding='same'))
# NN.add(Conv2D(filters=16, kernel_size=3, activation='relu', padding='same'))
NN.add(MaxPooling2D())

# Conv block 2 - note we increase filter dimension as we move
# further into the network. You can add more conv steps to
# each block to increase model capacity.
NN.add(Conv2D(filters=20, kernel_size=3, activation='relu', padding='same'))
# NN.add(Conv2D(filters=16, kernel_size=3, activation='relu', padding='same'))
NN.add(MaxPooling2D())

# Conv block 3 - The conv blocks should be ended with either a flatten
# layer or a global pooling layer. These transform the 2D layers to 1D
# to match the following dense layers.
NN.add(Conv2D(filters=30, kernel_size=3, activation='relu', padding='same'))

NN.add(GlobalAveragePooling2D())

# Fully connected block - flattening followed by dense and output layers
# NN.add(Flatten())
NN.add(Dense(20, activation='relu'))
NN.add(Dense(10, activation='softmax'))  # 10 target classes

NN.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'],
)
NN.summary()
NN.fit(X_train, y_train_cat, epochs=5, verbose=1, validation_split=0.25,
       callbacks=[
           keras.callbacks.ModelCheckpoint(
               'models/mnist.{epoch:02d}-{val_loss:.2f}.hdf5',
               save_best_only=True)
       ])  # track progress as we fit

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, NN.predict_classes(X_test))

This is a solid starting point for this problem, and we can likely do even better with some fine tuning. But we'll go ahead and move on to a CNN approach that's typically even more successful, **transfer learning**.

---
**Extended Optional Exercise**: adjust the hyperparameters in our CNN to try to improve on the baseline performance. You may want to try adjusting # of epochs, # of filters, # of conv blocks, # of fully connected layers and units in those layers, adding dropout, etc. 

It may be helpful to hold out some of the training data as a validation set to get continuous feedback on epoch performance while training (or even to use for early stopping!)    

---

### 4. Applying Transfer Learning: Pretrained Convolutional Base

#### Complete Pretrained Network

Recall that transfer learning leverages the useful fact that model parameters trained to solve a very general task can often be effectively applied to solve a more specialized task. In computer vision, a typical general task would be the 1000 class classification problem given by the large database **imagenet**, which includes a wide range of images of animals, objects, etc. The idea is that a model which learns how to recognize a diverse array of high-signal patterns in raw pixels can be adapted with some fine-tuning to many common computer vision tasks. Also, we save a massive amount of compute time by reusing knowledge of general patterns instead of learning them again from scratch.  

To that end, a very typical workflow in image classification problems is to take a **pretrained convolutional base** from an imagenet model, and simply add and train from scratch the **fully connected block** at the end of the network. You can think of this as a specialist, fully connected model learning how to predict based off of a set of features that were pre-extracted from the convolutional base. 

Many pretrained imagenet models exist and are publicly accessible, including:
* VGG16
* VGG19
* InceptionV3
* MobileNet

In this notebook, we'll focus on MobileNet because it's a solid, established standard and easy to use. Also, unlike the other pretrained models provided by Keras, MobileNet is designed to achieve a high accuracy with the smallest, fastest model. The largest model provided by Keras is VGG19 which takes up 549 MB when all the weights are saved.  MobileNet_V2 outperforms VGG19 with just 14 MB of weights.

But feel free to experiment! We'll first take a look at what the pretrained network can do on its own before jumping into the transfer learning part. Note that MobileNet follows a conventional structure of gradually reducing width and height while increasing filter depth over a series of convolutional blocks, to eventially terminate in flattening and fully connected layers. It's basically just a souped up version of our network trained from scratch! 

You can read more about MobileNet [here](https://arxiv.org/abs/1801.04381).

## MobileNet

![](img/vgg16_croped.png)

With that in mind, let's set up some utility functions for loading images from jpg / jpeg into (4 dimensional) numpy arrays that we can feed to a keras model. Then **we'll use keras utilities to load the pretrained MobileNet model** and check out some of the predictions it makes.

In [ ]:
from tensorflow.keras.applications import mobilenet_v2
from tensorflow.keras.preprocessing import image


def prepare_image(img_path):

    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = mobilenet_v2.preprocess_input(x)
    return x

In [ ]:
# This is all we need to load and use the full pretrained model!
model = mobilenet_v2.MobileNetV2(weights='imagenet',)

x = prepare_image('img/dog.jpeg')

out = model.predict(x)

print('Predicted:', mobilenet_v2.decode_predictions(out))

![doge](img/dog.jpeg)

Pomerian is close enough, I guess? Here's another one:

![sloth](img/sloth.jpg)

In [ ]:
x = prepare_image('img/sloth.jpg')

out = model.predict(x)

print('Predicted:', mobilenet_v2.decode_predictions(out))

Well... it's not wrong! The reason VGG can't give us a perfect label is likely that these exact animals/breeds don't occur as labels in imagenet. Let's take a look at the **VGG16 model summary** to see exactly what its structure looks like in keras.

In [ ]:
model.summary()

It's worth spending a good amount of time parsing through this summary to understand the structure (it's highly repetitive!), and that will leave us with a good foundation for moving to the next step for transfer learning: extracting and freezing the convolutional base (all layers prior to flattening), **then adding new fully connected layers to the base**, then training. 

#### Transfer Learning to Build on the Pretrained Base

Once we've selected a specialized predictive task (say the MNIST digit recognition problem) we can apply transfer learning with the following 3 steps immediately prior to training in a normal manner:
1. Initialize the base (VGG16) model but **exclude the "top" layers** (flatten + fully connected)
2. **Freeze the base layers** to be untrainable during the gradient descent process
3. **Add new "top" layers** to the base model (randomly initialized weights) 

Note that we freeze the base layers both because we don't want to have to take the time to try to update them when they are already good signal extractors, and because trying to update them naively would actually destroy them! With the new random top layer weights, the model will initially make horribly wrong predictions that will back-propagate huge gradient signals, easily destroying the base of the network. In more advanced cases, it's possible to train the model with a frozen base until the new layers become stable, then step back into the end of the convolutional base to fine-tune those layers with further iterations.

Below are the 3 steps in action in keras. Note the use of the **functional API** syntax, which is easier to work with than the sequential API when applying transfer learning.

In [ ]:
from tensorflow.keras.models import Model

# note we exclude the final dense layers by setting include_top=False,
# and add new ones to train from scratch below
base_model = mobilenet_v2.MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3)) 
 
# Freeze convolutional layers
for layer in base_model.layers:
    layer.trainable = False    

# Establish new fully connected block
x = base_model.output
x = Flatten()(x) # flatten from convolution tensor output 
x = Dense(100, activation='relu')(x) # number of layers and units are hyperparameters, as usual
x = Dense(50, activation='relu')(x)
predictions = Dense(5, activation='softmax')(x) # should match # of classes predicted

# define formal model object to train and compile it as usual
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Then you would just train like normal.

```python
# i.e. if we had training images and our own labels, we could run
model.fit(X_train,y_train)
```

Here are [more examples of keras transfer learning](https://keras.io/applications/) with modern pretrained CNNs. Check out the documentation specific to the model(s) you want to use.

---
**Extended Optional Exercise**: apply the above transfer learning framework to the MNIST digits problem to see if you can improve on our current score above. 

This will be time consuming due to the complexity of the VGG base, so it may be more paleatable to run it on google colab (with GPU) or on an AWS instance (e.g. a p2.xlarge GPU instance with the deep learning AMI). Also, consider the feature pre-extraction step discussed below as a possible speedup.   

---

### 5. Appendix 1: Additional Considerations for Computer Vision Problems

#### How much data do you need? (image augmentation)

More! More data is always better :)

However, with this bottleneck approach, you don't necessarily need as much data as you would otherwise. 200-1000 representitive images of each class may give good results. This is due to 2 reasons --
* By applying transfer learning, we are effectively using a much larger dataset (imagenet) to establish good base model parameters 
* We can use **image augmentation** to synthetically increase our number of training samples

In image augmentation, we manipulate the raw training samples to preserve the content of the objects we're detecting but change object positioning within the image, image quality, etc. to generate new artificial samples. The cat pictures below should illustrate this clearly. This strategy is often an excellent way to force a model to learn more robust/generalizable object patterns that are relatively independent of position and hyper-specific quality/coloring. For example, we can encourage a model to recognize a cat by finding fur, whiskers, and fluffy ears anywhere in a photo, rather than overfitting to the cat's presence right in the middle of the photo.   

![](img/DataAugmentation.png)

Possible augmentations include:
* Scale
* Rotation
* Skew
* Flips
* Color tinting
* Blur
* Crop
* And more! ... As long as you do not destroy the info you are trying to represent.

[Check out this](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html) for more with keras!

#### Is there a way to more efficiently use the pre-trained base? (feature extraction as preprocessing)

Yes! If you think about the *frozen* convolutional base, you realize that the exact same deterministic processing steps will happen every time you feed the *same* image sample as input to the base. If you're training your model for many epochs, this will amount to a huge number of repetitious, unecessary calculations. 

To work around this inefficiency, we can treat the pretrained base part of the model as a **feature preprocessing step** that we run *once* to extract convolutional features that will be fed as flat feature inputs to a new, fully connected model. Naturally, this can dramatically speed up the training process. Check out the first section of this [excellent notebook](https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/5.3-using-a-pretrained-convnet.ipynb) from Francois Chollet for more explanation and detailed code to accomplish this. 

#### How can I deal with the huge RAM cost of image data? (out of core processing with data generators)

When working with image data, you'll realize that large 4D arrays can very quickly eat up your entire RAM, making it extremely difficult to work with large quantities of images in a static array. However, neural networks' **mini-batch gradient descent** training process is naturally amenable to **out of core learning**, where we don't hold all training samples in memory simultaneously. Instead, we can keep images saved to disk and **load each small training batch to RAM on the fly** before discarding it and moving to the next batch. 

Instead of implementing the logic for this method from scratch, we can leverage existing keras utilities: the **ImageDataGenerator** object and **fit_generator** model methods. You will almost always see these used in practical keras applications, and you can learn more about how to use them from the same [Chollet notebook](https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/5.3-using-a-pretrained-convnet.ipynb). 

As a final note on ImageDataGenerator, it's also very useful as a way to perform image augmentation on the fly. You can specify parameters that cause the generator to derive a different synthetic sample from the real one every time it gets loaded. The main downside to this approach is that it makes it impossible to do the convolutional base preprocessing shortcut described above, since every augmented image will be a technically different array that needs to be processed by the base from scratch once that array gets generated.  

### 6. Appendix 2: More Deep Learning Architectures for Computer Vision

#### Auto-Encoders

Auto-encoders are sort of like doing matrix factorization and are used only for unsupervised learning. They use either shallow or deep networks. With certain architectures, you could also make a recommender system out of an auto-encoder. The difference from traditional matrix factorization is that we can get non-linearity through our choice of activation function in the output layer. 

The matrix decompositions and feature extraction methods we've seen (e.g. SVD and PCA) can be considered linear special cases of auto-encoders. Using a neural network, though, we can add non-linear reduction methods. Because they are feature engineering methods (e.g. like PCA), word-embeddings (e.g. Word2Vec) can be considered auto-encoders. 

Applications: 
* Word-embeddings (e.g. Word2Vec)
* Non-linear feature extraction
* When using a deep network, we get **hierarchical data reduction**
* Recommender systems
* Outlier detection (remember, the lower-rank approximation of a matrix is considered de-noised, removing outliers); take the difference between A and A' (where A' is the matrix factorization) to find outliers 

Benefits include:
* Non-linearity via activation functions and architecture
* Easier to map out-of-sample points to our new space

![](img/Autoencoder.png)

#### Variational Auto-Encoder
![](http://kvfrans.com/content/images/2016/08/vae.jpg)
In a VAE, the "loss function is modified to impose a specific structure on the hidden layer" (Aggarwal, pg. 82). That is, we can specify the distribution that hidden variables are drawn from. A benefit of imposing this constraint is that we can get rid of the encoder after training and use only the decoder. If we feed draws from the normal distribution (say, if that's what our constraint was), we then feed it to the decoder to get new samples. 

VAE are most commonly used in image processing. They are used to create new, realistic images from a corpus of images. 

#### Generative Adversarial Networks

To start, we have to create two types of models: 

1. **Discriminative**: Think logistic regression for classification. Here's we're trying to estimate a conditional probability of a label or class (e.g. P(y|X), or what's the probability that this image is a dog, given that it has these features). These are supervised models. 

2. **Generative**: In this case, we're trying to estimate a joint probability (think Naive Bayes). We can translate from a joint probability (e.g. P(X,y)) back to a conditional probability using Bayes' theorem. You can use the joint probability to create a prior distribution that you sample from, where these samples are _generated_ samples of interest. These models can be supervised or unsupervised. 

A GAN in deep learning uses one of each of these types of models. First, we use a generative model to come up with new samples/examples that are close to our corpus (e.g. new images of faces from a corpus of real faces). Then the discriminative model, which was trained on our _labeled_ corpus of real images plus some fakes, will try to guess whether generated samples are real or not. The discriminative model will also be trained on real images. We pit them against each other until there's an equilibrium. 

![](img/GAN.jpg)

[Really cool example of Adversarial Networks in action](http://carpedm20.github.io/faces/)

#### Stack GANs
Gans can be stacked with awesome results
![](http://i.imgur.com/SGzE7vI.jpg)

https://arxiv.org/pdf/1612.03242v1.pdf

#### Image analogy

![](https://raw.githubusercontent.com/awentzonline/image-analogies/master/examples/images/sugarskull-analogy.jpg)
![](https://raw.githubusercontent.com/awentzonline/image-analogies/master/examples/images/image-analogy-explanation.jpg)
![](https://raw.githubusercontent.com/awentzonline/image-analogies/master/examples/images/trump-image-analogy.jpg)

https://github.com/awentzonline/image-analogies

#### New architectures are being published every day. So much to read!

* [Curated List of Deep Learning papers](https://github.com/ChristosChristofidis/awesome-deep-learning)
* [Good reddit post for keeping up with the latest research](https://www.reddit.com/r/MachineLearning/comments/6d7nb1/d_machine_learning_wayr_what_are_you_reading_week/)
